CAPTURA DE IMÁGENES ESTÉREO SINCRONIZADAS DESDE DOS CÁMARAS CON HILOS

Funcionalidad principal:
- Conexión simultánea de dos cámaras vía IP usando DroidCam
- Visualización en tiempo real de ambas cámaras en una única ventana, mostradas lado a lado
- Captura sincronizada de pares de imágenes estéreo al presionar la tecla "c"
- Almacenamiento de las imágenes con nombres numéricamente sincronizados en carpetas separadas (una por cámara)
- Detección automática del siguiente índice disponible para evitar sobrescribir archivos
- Finalización segura del programa al presionar "q" o "ESC"

Implementación técnica:
- StereoCamera (clase principal):
    - init():
        - Establece la conexión con ambas cámaras IP
        - Define los directorios de guardado y la extensión de imagen
        - Crea las carpetas si no existen
        - Inicializa variables compartidas (como los frames y locks)
        - Prepara los hilos (threading.Thread) para capturar video en paralelo
        - Calcula el índice inicial a partir de los archivos existentes
    - capture_R() y capture_L():
        - Corren en paralelo para capturar continuamente imágenes de cada cámara
        - Guardan los frames en memoria protegidos por threading.Lock
    - getNextIndex():
        - Escanea ambos directorios para encontrar el número de archivo más alto
        - Devuelve el próximo índice disponible para nombrar imágenes nuevas
    - saveImages():
        - Guarda un par de imágenes (izquierda y derecha) con el mismo número de índice
        - Informa si la operación fue exitosa
    - run():
        - Inicia los hilos de captura
        - Muestra una ventana combinada con ambas imágenes lado a lado
        - Permite al usuario capturar pares de imágenes con "c"
        - Permite salir del programa con "q" o "ESC", ejecutando limpieza segura
    - cleanup(): 
        - Detiene los hilos de captura
        - Libera las cámaras y destruye la ventana abierta

Modo de ejecución:
- Definición manual de las IPs de ambas cámaras 
- Definición de directorios donde se guardarán las imágenes capturadas
- Instanciación de un objeto StereoCamera
- Llamado a run() para iniciar la captura y visualización

Autores:
- Belda Martínez, Marcos
- Espert Cornejo, Ángela
- Francés Llimerá, Lourdes

In [5]:
import os          # File and directory operations
import cv2         # OpenCV library for video capture and image processing
import re          # Regular expressions for filename matching
import numpy as np # Efficient matrix and array operations
import threading   # Enables multithreaded execution for concurrent tasks
import time        # Time handling (e.g., delays, timestamps)

In [6]:
class StereoCamera:
    """
    Class to manage stereo camera input, display, and saving of synchronized images.
    """
    
    def __init__(self, ip_L, ip_R, dir_L, dir_R, ext):
        """
        Initializes stereo camera setup, including connection to both IP cameras,
        save directories, and thread-safe capture configuration.
        """
        # Initialize camera connections (port 0000)
        self.capture_L = cv2.VideoCapture(f"http://{ip_L}:8080")
        self.capture_R = cv2.VideoCapture(f"http://{ip_R}:8080")
            
        # Validate left camera connection
        if not self.capture_L.isOpened():
            print(f"ERROR: Couldn't connect to left camera at {ip_L}")
            self.capture_R.release()
            exit()

        # Validate right camera connection
        if not self.capture_R.isOpened():
            print(f"ERROR: Couldn't connect to right camera at {ip_R}")
            self.capture_L.release()
            exit()
        
        # Set output directories and image extension
        self.dir_L = dir_L
        self.dir_R = dir_R
        self.extension = ext.lower()
        
        # Display window name
        self.window = "Stereo View"
        
        # Ensure directories exist
        os.makedirs(self.dir_L, exist_ok = True)
        os.makedirs(self.dir_R, exist_ok = True)
        
        # Shared frame variables and locking mechanism
        self.frame_L = None
        self.frame_R = None
        self.lock = threading.Lock()
        self.thread_stop = False
        
        # Threads for concurrent image capture
        self.thread_L = threading.Thread(target = self.leftCapture)
        self.thread_R = threading.Thread(target = self.rightCapture)
        
        # Initialize the file index based on existing files
        self.idx = self.getNextIndex()
        
    def getNextIndex(self):
        """
        Scans both directories for existing images and returns the next available index
        for consistent naming of saved stereo pairs.
        """
        patternName = re.compile(r'^(\d{6})\.' + self.extension + r'$', re.IGNORECASE)
        maxIndex = -1
        
        # Search for higher index in both directories
        for directory in [self.dir_L, self.dir_R]:
            for filename in os.listdir(directory):
                patternMatch = patternName.match(filename)
                # Search pattern coincidence
                if patternMatch:
                    index = int(patternMatch.group(1))  # Gets image number
                    if index > maxIndex:
                        maxIndex = index
                        
        return maxIndex + 1                             # Returns next available index
    
    def saveImages(self, frame_L, frame_R):
        """
        Saves the current stereo image pair to disk using synchronized naming.
        """
        filename_L = os.path.join(self.dir_L, f"{self.idx:06d}.{self.extension}")
        filename_R = os.path.join(self.dir_R, f"{self.idx:06d}.{self.extension}")
        
        if cv2.imwrite(filename_L, frame_L) and cv2.imwrite(filename_R, frame_R):
            print(f"Images saved: {filename_L} | {filename_R}")
            self.idx += 1
        else:
            print("ERROR: Couldn't save the images")
        
    def leftCapture(self):
        """
        Continuously captures frames from the left camera in a separate thread.
        """
        while not self.thread_stop and self.capture_L.isOpened():
            ret, frame = self.capture_L.read()
            if ret:
                with self.lock:
                    self.frame_L = frame.copy()  
        
    def rightCapture(self):
        """
        Continuously captures frames from the right camera in a separate thread.
        """
        while not self.thread_stop and self.capture_R.isOpened():
            ret, frame = self.capture_R.read()
            if ret:
                with self.lock:
                    self.frame_R = frame.copy()  
                    
    def run(self):
        """
        Main control loop: displays stereo images and allows user to save them.
        """
        print("Press 'c' to capture image pair. Press 'q' or 'ESC' to quit.")
        cv2.namedWindow(self.window, cv2.WINDOW_NORMAL)
        
        # Start capture threads
        self.thread_L.start()
        self.thread_R.start()
        
        # Allow time for camera warm-up
        time.sleep(1)
        
        while True:
            # Capture one image from each camera
            with self.lock:
                img_L = self.frame_L
                img_R = self.frame_R
                
            if img_L is None or img_R is None:
                print("ERROR: No image captured.")
                continue
                
            # Resize images to same size
            h = min(img_L.shape[0], img_R.shape[0])
            w = min(img_L.shape[1], img_R.shape[1])
            resized_L = cv2.resize(img_L, (w, h))
            resized_R = cv2.resize(img_R, (w, h))
            
            # Combine both images side-by-side
            combined = np.hstack((resized_L , resized_R))
            cv2.imshow(self.window, combined)
            
            # Wait for key to be pressed
            key = cv2.waitKey(1) & 0xFF
            if key == ord('c'):
                self.saveImages(img_L, img_R)
            elif key in (27, ord('q')):  # ESC or 'q'
                print("Closing program...")
                break
            
        self.cleanup()
        
    def cleanup(self):
        """
        Safely stops threads, releases camera resources, and closes display window.
        """
        self.thread_stop = True
        self.thread_L.join()
        self.thread_R.join()
        self.capture_L.release()
        self.capture_R.release()
        cv2.destroyAllWindows()
        print("Resources released.")           

In [7]:
if __name__ == "__main__":
    # Define IP addresses of the stereo camera pair
    ip_L = "192.168.1.132"
    ip_R = "192.168.1.129"
    
    # Output directories for images
    dir_L = "stereo_images_L"
    dir_R = "stereo_images_R"
    
    # Image file extension
    extension = "bmp"

    # Create and run stereo camera capture
    stereo_cam = StereoCamera(ip_L, ip_R, dir_L, dir_R, extension)
    stereo_cam.run()

Press 'c' to capture image pair. Press 'q' or 'ESC' to quit.
Images saved: stereo_images_L\000000.bmp | stereo_images_R\000000.bmp
Images saved: stereo_images_L\000001.bmp | stereo_images_R\000001.bmp
Images saved: stereo_images_L\000002.bmp | stereo_images_R\000002.bmp
Images saved: stereo_images_L\000003.bmp | stereo_images_R\000003.bmp
Images saved: stereo_images_L\000004.bmp | stereo_images_R\000004.bmp
Images saved: stereo_images_L\000005.bmp | stereo_images_R\000005.bmp
Images saved: stereo_images_L\000006.bmp | stereo_images_R\000006.bmp
Images saved: stereo_images_L\000007.bmp | stereo_images_R\000007.bmp
Images saved: stereo_images_L\000008.bmp | stereo_images_R\000008.bmp
Images saved: stereo_images_L\000009.bmp | stereo_images_R\000009.bmp
Images saved: stereo_images_L\000010.bmp | stereo_images_R\000010.bmp
Images saved: stereo_images_L\000011.bmp | stereo_images_R\000011.bmp
Images saved: stereo_images_L\000012.bmp | stereo_images_R\000012.bmp
Images saved: stereo_images_L